### Imports

In [1]:
import pandas as pd
import numpy as np
import cv2
import math
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.neighbors import NearestNeighbors
import os
from PIL import Image
import numpy as np
import pywt
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

### Hacer resize y guardar una imagen

In [2]:
def resize_and_save_img(src, destination_path):
    original_img = cv2.imread(src)
    old_image_height, old_image_width, channels = original_img.shape
    new_image_width = 60        
    new_image_height = 60
    color = (255,255,255)

    result = np.full((new_image_height, new_image_width, channels), color, dtype=np.uint8)

    x_center = (new_image_width - old_image_width) // 2
    y_center = (new_image_height - old_image_height) // 2

    # Centrar imagen
    result[y_center:y_center+old_image_height, x_center:x_center+old_image_width] = original_img

    Image.fromarray(result).save(destination_path)

### Unificar data de Train y Test

Se realizará split propio posteriormente (70% TRAIN y 30% TEST)

In [44]:
def generate_new_data():
    train_dir = "Data/Train/"
    test_dir = "Data/Test/"
    destination_dir = "Data_preprocesada/"

    for class_dir in os.listdir(train_dir):
        for train_img in os.listdir(train_dir+class_dir):
            resize_and_save_img(f"{train_dir}{class_dir}/{train_img}", f"{destination_dir}{class_dir}/{train_img}")

    test_info = pd.read_csv("Data/Test.csv")
    for i, test_img in enumerate(sorted(os.listdir(test_dir))):
        resize_and_save_img(f"{test_dir}{test_img}", f"{destination_dir}{test_info.ClassId[i]}/{test_img}")

generate_new_data()

In [4]:
def get_vector_from_image(image, iterations):
    LL, (LH, HL, HH) = pywt.dwt2(image, 'haar')
    for _ in range(iterations - 1):
        LL, (LH, HL, HH) = pywt.dwt2(LL, 'haar')
    return LL.flatten()

#### Obtener data (estratificada y no estratificada)

In [6]:
def get_data(src_dir, iterations, number_of_classes):
    x_by_class = [[] for _ in range(number_of_classes)]
    x = []
    for class_dir in os.listdir(src_dir):
        for train_img in os.listdir(src_dir + class_dir):
            image_path = f"{src_dir}{class_dir}/{train_img}"
            img = Image.open(image_path)
            fv = get_vector_from_image(img, iterations)
            x_by_class[int(class_dir)].append(fv)
            x.append(fv)
    return np.asarray(x_by_class), np.asarray(x)[np.random.permutation(len(x))] # stratified, non stratified

stratified_data, non_stratified_data = get_data(src_dir="Data_preprocesada/", iterations=5, number_of_classes=10)

In [7]:
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a))
    return a[p], b[p]

### Cantidad de muestras por categoria

In [8]:
counts = { }
for i in range(len(stratified_data)):
    counts[i] = len(stratified_data[i])

counts

{0: 195,
 1: 2158,
 2: 2466,
 3: 1599,
 4: 2239,
 5: 2210,
 6: 522,
 7: 1640,
 8: 1648,
 9: 1560}

### Cantidad de caracteristicas por muestra

In [9]:
print(len(stratified_data[0][0]))

120


### K-fold Cross Validation Estratificado

In [134]:
def stratified_k_fold_cross_validation(stratified_data, number_of_folds):
    categories = len(stratified_data)

    data_by_fold = [] #Cada Fold tendrá un diccionario de X_train,X_test,y_train, y_test
    
    for fold in range(number_of_folds):
        X_train = []
        X_test = []
        y_train = []
        y_test = []

        for category in categories:
            offset_size = math.round(stratified_data[category]/number_of_folds)
            upper_bound = fold
            lower_bound = fold
            
            if upper_bound>len(stratified_data[category]):
                upper_bound = len(stratified_data[category])

            #0 1 2 3 4 5 6 7 8 9
            #0 1 2 3 4 5 6 7 8 9

11


10/4 = 3

3 3   1

11/4 = 3

3 3 3 2
10/4 = 3 3 3 1
9/4 = 2 2 2 3


3

In [10]:
print("hola")

hola


### Split de Train y Test

In [135]:
def non_stratified_split_train_test_classes(X_vectors, labels):
    categories = np.unique(labels).tolist()

    X_train = []
    X_test = []
    y_train = []
    y_test = []
    
    for i in range(len(categories)):
        labels_i = [i for _ in range(len(X_vectors[i]))]
        Xi_train, Xi_test, yi_train, yi_test = train_test_split(X_vectors[i], labels_i, test_size=0.3)

        X_train += Xi_train
        X_test += Xi_test
        y_train += yi_train
        y_test += yi_test

    return np.asarray(X_train), np.asarray(X_test), np.asarray(y_train), np.asarray(y_test)


ns_X_train, ns_X_test, ns_y_train, ns_y_test = non_stratified_split_train_test_classes(X_vectors, labels)
ns_X_train, ns_y_train = unison_shuffled_copies(X_train, y_train)
ns_X_test, ns_y_test = unison_shuffled_copies(X_test, y_test)

### K-fold Cross Validation no Estratificado

In [3]:
def non_stratified_k_fold_cross_validation(X, y, number_of_folds):
    for train_index, test_index in kf.split(X):
        print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

#### KNN Classifier

In [138]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)